In [6]:
x = deque(maxlen=3)
x

deque([])

In [7]:
len(x)

0

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from plot import *

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

path = "C:/Users/cc/Desktop/CedAlgo/AlgoTrading/database/"

def get_data(Id , interval):
    engine = sqlalchemy.create_engine('sqlite:///'+path+'database_{}.db'.format(interval))
    
    data = pd.read_sql(Id+'USDT' ,engine)
    data.set_index('time' , inplace=True)
    data['volume'] = pd.to_numeric(data['volume'])
    data = data[['open', 'high', 'low' , 'close' , 'volume', 'symbol', 'close_time']]
    return data

In [2]:
data = get_data("TWT", "1d")
data = data["2022":"2022"]

In [3]:
def getEvents(gRaw, h):
    tEvents, sPos, sNeg = [], 0, 0
    diff = gRaw.diff()
    for i in diff.index[1:]:
        sPos, sNeg = max(0, sPos + diff.loc[i]), min(0, sNeg + diff.loc[i])
        if sNeg < - h:
            sNeg = 0; tEvents.append(i)
        elif sPos > h:
            sPos = 0; tEvents.append(i)
    return pd.DatetimeIndex(tEvents)

In [21]:
pos = getEvents(data.close, 0.1)
cumsum_filter = data.loc[pos]

candle = add_candle(data)
cumsum_mark = add_mark(x = cumsum_filter.index, y = cumsum_filter.close, color = 'black', name = 'cumsum')

layout = go.Layout(
        height = 600 , width = 1400,
        template="simple_white",
        margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
        xaxis = dict(rangeslider_visible = False)
        )

fig = go.Figure(
    data = [candle, cumsum_mark],
    layout = layout
)

fig

In [ ]:
from scipy.linalg import inv
import numpy as np

def kalman_filter(mu, sigma, y, A, B, C, Q, R):
    # mu : état initial
    # sigma : covariance de l'état initial
    # y : observation
    # A : matrice de transition d'état
    # B : matrice d'entrée de commande
    # C : matrice d'observation
    # Q : covariance du bruit de processus
    # R : covariance du bruit de mesure

    # Calcul de la prédiction d'état
    mu_hat = np.dot(A, mu) + np.dot(B, u)
    sigma_hat = np.dot(np.dot(A, sigma), A.T) + Q

    # Calcul de l'innovation et de sa covariance
    innov = y - np.dot(C, mu_hat)
    innov_cov = np.dot(np.dot(C, sigma_hat), C.T) + R

    # Calcul du gain de Kalman
    K = np.dot(np.dot(sigma_hat, C.T), inv(innov_cov))

    # Mise à jour de l'état et de sa covariance
    mu_new = mu_hat + np.dot(K, innov)
    sigma_new = np.dot(np.eye(len(mu)) - np.dot(K, C), sigma_hat)

    return mu_new, sigma_new


In [ ]:
#   SNIPPET 3.1 DAILY VOLATILITY ESTIMATES

def getDailyVol(close, span0 = 100):
    # daily vol, reindex to close
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days = 1))
    df0 = df0[df0 > 0]
    df0 = pd.Series(close.index[df0 - 1], index = close.index[close.shape[0] - df0.shape[0] :])
    df0 = df0.ewm(span = span0).std()
    return df0


In [ ]:
#   SNIPPET 3.2 TRIPLE-BARRIER LABELING METHOD


def applyPtSlonTl(close, events, ptSl, molecule):
    # apply stop loss/profit taking, if ti takes place vefore t1 ( end of event)
    events_ = events.loc[molecule]
    out = events_[['t1']].copy(deep = True)
    if ptSl[0] > 0:
        pt = ptSl[0] * events_['trgt']
    else:
        pt = pd.Series(index = events.index)
    
    if ptSl[1] > 0:
        sl = -ptSl[1] * events_['trgt']
    else:
        sl = pd.Series(index = events.index) * events   # NaNs
    
    for loc, tl in events_['t1'].fillna(close.index[-1]).iteritems():
        df0 = close[loc:t1] # path prices
        df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']  # path returns
        
        out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()     # earliest stop loss
        out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()     # earliest profit taking
        

In [ ]:
import numpy as np

def cumsum_filter(signal, threshold):
    """
    Applique le filtre de cumsum à un signal donné avec un seuil donné.
    """
    signal_cumsum = np.cumsum(signal)
    signal_cumsum_diff = np.diff(signal_cumsum)
    filtered_signal = np.zeros_like(signal)
    filtered_signal[0] = signal[0]
    for i in range(1, len(signal)):
        if signal_cumsum_diff[i-1] > threshold:
            filtered_signal[i] = filtered_signal[i-1]
        else:
            filtered_signal[i] = signal[i]
    return filtered_signal
